# Spectrum Class GAIA-NIR



In [1]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt

class Spectrum:
    """
    Class to manage spectrum data and operations.
    """

    def __init__(self):
        """
        Initialize spectrum attributes.
        """
        self.wavelength = np.array([])
        self.flux = np.array([])
        self.original_wavelength = np.array([])
        self.original_flux = np.array([])

    def load_spectrum(self, file_name):
        """
        Load input spectrum from a file and parse data.
        """
        print(f"Loading spectrum from {file_name}")
        try:
            data = np.loadtxt(file_name, skiprows=1)
            self.wavelength = data[:, 0]
            self.flux = data[:, 1]
            print("Spectrum loaded correctly.") 
        except Exception as e:
            print(f"Error loading spectrum: {e}")

    def reference_spectrum(self):
        """
        Store a reference copy of the original spectrum before modifications.
        """
        self.original_wavelength = self.wavelength.copy()
        self.original_flux = self.flux.copy()
        print("Reference spectrum saved.")

    def resample_and_convolve(self, sigma=5.0, verbose=True):
        """
        Resample the spectrum onto a new wavelength grid and convolve with a Gaussian.

        Args:
            sigma (float): Standard deviation for Gaussian convolution.
            verbose (bool): If True, prints the flux conservation check.
        """
        
        new_wavelength_grid = np.linspace(min(self.wavelength), max(self.wavelength), 1000)

       
        self.reference_spectrum()

        # Compute total flux before convolution
        # Integrate using trapezoidal rule
        initial_flux = np.trapz(self.flux, self.wavelength)  

        interpolator = interp1d(self.wavelength, self.flux, kind="linear", bounds_error=False, fill_value="extrapolate")
        resampled_flux = interpolator(new_wavelength_grid)

        convolved_flux = gaussian_filter1d(resampled_flux, sigma, mode='reflect')

        # Compute total flux after convolution
        final_flux = np.trapz(convolved_flux, new_wavelength_grid)  # Integrate over new grid

        self.wavelength = new_wavelength_grid
        self.flux = convolved_flux

        # Compute flux conservation ratio to compareee
        conservation_ratio = final_flux / initial_flux if initial_flux != 0 else 0

        # Print conservation check if verbose=True
        if verbose:
            print("Resampling and convolution completed.")
            print(f"Flux before convolution: {initial_flux:.6f}")
            print(f"Flux after convolution: {final_flux:.6f}")
            print(f"Flux conservation ratio: {conservation_ratio:.6f}")

        return self 
    
    def plot_comparison(self):
        """
        Plot the spectrum before and after convolution for visualization.
        """
        if self.original_wavelength is None or self.original_flux is None:
            print("Original spectrum not found. Ensure to call resample_and_convolve first.")
            return
        
        plt.figure(figsize=(15, 10))
        plt.plot(self.original_wavelength, self.original_flux, label="Original Spectrum", color="pink", linestyle='--')
        plt.plot(self.wavelength, self.flux, label="Convolved Spectrum", color="red")
        plt.xlabel("Wavelength")
        plt.ylabel("Flux")
        plt.legend()
        plt.grid(True)
        plt.show()    

    def convert_units(self):
        print("Converting units")

    def rescale_flux(self):
        print("Rescaling flux levels")

    def radial_velocity_shift(self, verbose=True):
        if verbose:
            print("Applying radial velocity shift")

    def resample_stochastic(self, verbose=True):
        if verbose:
            print("Resampling spectrum for stochastic process")

    def generate_noise(self, verbose=True):
        if verbose:
            print("Adding noise to spectrum")

    def save_spectrum(self, output_file):
        """
        Save the processed spectrum to a file.
        """
        print(f"Saving spectrum to {output_file}")
